# Chapter 9: Pythonic Objects

Python's data model allows user-defined types to behave as naturally as the built-in types. This can be accomplished without inheritance, in the spirit of [duck typing](https://en.wikipedia.org/wiki/Duck_typing): you implement the methods needed for your objects to behave as expected.

This chapter shows how to implement several special methods that are commonly seen in Python objects of many different types.

We will see how to:

* Support the built-in functions that produce alternative object representation.

* Implement an alternative constructor as a class method.

* Extend the format mini-language used by the `format()` built-in and `str.format()` method.

* Provide read-only access to attributes.

* Make an object hashable for use in sets and `dict` keys.

* Save memory by using `__slots__`.

Through examples we will also examine:

* How and when to use the `@classmethod` and `@staticmethod` decorators.

* Private and protected attributes in Python: usage, conventions, and methods.

## Object Representations

Recall that Python has two standard ways of getting a string representation from an object:

1. `repr()`: Return a string representing the object as the **developer** wants to see it.

2. `str()`: Return a string representing the object as the **user** wants to see it.

The special methods `__repr__` and `__str__` support `repr` and `str()`.

There are two additional special methods to support alternative string representations:

3. `__bytes__`: Return the byte string representation of the object (invoked by `bytes()`).

4. `__format__`: Display object using special formatting codes (invoked by `format()` and `str.format()`.

## Vector Class Redux

To demonstrate different ways of generating object representations, we will extend the vector class from [chapter 1]() to two dimensions.

In [21]:
# Example 9-2. vector2d_v0.py: Implementation of Vector2d using only special methods

from array import array
import math

class Vector2d:
    typecode = "d" # Used when converting Vector2d instances to/from bytes
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self): # Makes Vector2d iterable, allows unpacking
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
        def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    

## An Alternative Constructor

Since we can export `Vector2d` as bytes, it makes sense to implement functionality to go the other way; we need a method to import `Vector2d` from a binary sequence.

In [25]:
# Example 9.3. vector2d_v1.py: Implementation of Vector2d allowing conversions from bytes to Vector2d instance.

from array import array
import math

class Vector2d:
    typecode = "d" # Used when converting Vector2d instances to/from bytes

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self): # Makes Vector2d iterable, allows unpacking
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)


## `classmethod` Versus `staticmethod`

`classmethod` is used to define a method that operates on the class and not on instances. 

`classmethod` changes the way the method is called, so it **receives the class itself as the first argument, instead of an instance**.

Its most common use is for alternative constructors, like `frombytes` in example 9-3. Note how the last line uses the `cls` argument by invoking it to build a new instance.

By convention, the first parameter of a class method should be named `cls`.

The `staticmethod` decorator changes a method so that it receives no special first argument.

**In essence, a static method is just like a plain function that happens to live in a class body, instead of being defined at the module level**.

The next example contrast the operation of `classmethod` and `staticmethod`.

In [32]:
# Example 9-4. Comparing behavious of classmethod and staticmethod.

class Demo:
    """
    klassmeth returns all positional arguments.
    statmeth does the same.
    """
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args

# No matter how it is invoked, `klassmeth` receives the Demo class as first argument
print(Demo.klassmeth())
print(Demo.klassmeth("spam"))

# `statmeth` behave like a plain function
print(Demo.statmeth())
print(Demo.statmeth("spam"))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


Ramalho is of the opinion that `@staticmethod` is of little practical use. He argues that if you want a function that doesn't interact with the class, simply define it in the module.

Ramalho provides [this blog post](https://julien.danjou.info/guide-python-static-class-abstract-methods/) as a counter-argument to his own viewpoint.

## Formatted Displays

TODO